In [ ]:
import argparse
import networkx as nx
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.contrib.eager as eager
import my
import gnn_lg
import objectives
import partition

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [ ]:
args = argparse.Namespace()
args.activation = 'relu'
args.depth = 10
args.device = '/cpu:0'
# args.device = '/device:GPU:0'
# args.graph = 'soc-Epinions1'
# args.graph = 'soc-Slashdot0811'
# args.graph = 'soc-Slashdot0902'
args.graph = 'barabasi-albert-100-3'
args.lambda_node = 0.0
args.lambda_edge = 0.0
args.lambda_entropy = 0.0
args.n_features = 16
args.n_iterations = 50
args.n_machines = 10
args.n_samples = 5
# args.objective = 'CrossEntropy'
args.objective = 'OraclePolicyGradient'
# args.objective = 'PolicyGradient'
# args.partitioner = ''
args.partitioner = 'GreedyVertexCut'
# args.partitioner = 'RandomVertexCut'
args.radius = 3

keys = sorted(vars(args).keys())
run_id = 'ipynb-' + '-'.join('%s-%s' % (key, str(getattr(args, key))) for key in keys if key != 'device')
writer = tf.contrib.summary.create_file_writer('runs/' + run_id)
writer.set_as_default()

In [ ]:
if args.graph.startswith('barabasi-albert'):
    n, m = tuple(map(int, args.graph.split('-')[-2:]))
    g = nx.barabasi_albert_graph(n, m)
else:
    g = my.read_edgelist('data/' + args.graph, fill=True)

In [ ]:
import imp
gnn_lg = imp.reload(gnn_lg)
objectives = imp.reload(objectives)

In [ ]:
with tf.device(args.device):
    lambda_tuple = (args.lambda_node, args.lambda_edge, args.lambda_entropy)
    if args.partitioner:
        partitioner = getattr(partition, args.partitioner)
        if args.objective == 'CrossEntropy':
            objective = objectives.CrossEntropy(g, partitioner, args.n_machines, *lambda_tuple)
        elif args.objective == 'OraclePolicyGradient':
            kwargs = {'node' : True}
            objective = objectives.OraclePolicyGradient(g, partitioner, args.n_machines, *lambda_tuple, **kwargs)
    else:
        objective = getattr(objectives, args.objective)(g, args.n_machines, *lambda_tuple)
            
    features = (1,) + (args.n_features,) * args.depth
    activation = getattr(tf.keras.activations, args.activation)
    network = gnn_lg.GNN(g, features, args.n_machines, args.radius, activation, args.device)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e1)

In [ ]:
global_step = tf.train.get_or_create_global_step()
for i in range(args.n_iterations):
    global_step.assign_add(1)
    with tf.contrib.summary.record_summaries_every_n_global_steps(1):
        with eager.GradientTape() as tape:
            z = network(None, training=True)
            objective_train = objective(z, args.n_samples, training=True)
        gradients = tape.gradient(objective_train, network.variables)
        optimizer.apply_gradients(zip(gradients, network.variables), global_step=tf.train.get_or_create_global_step())

        tf.contrib.summary.scalar('objective-train', objective_train)

        objective_test, r, b_node, b_edge, entropy = objective(z, args.n_samples)
        tf.contrib.summary.scalar('objective', objective_test)
        tf.contrib.summary.scalar('replication-factor', r)
        tf.contrib.summary.scalar('node-balancedness', b_node)
        tf.contrib.summary.scalar('edge-balancedness', b_edge)
        tf.contrib.summary.scalar('entropy', entropy)

    if (i + 1) % 1 == 0:
        prefix = '0' * (len(str(args.n_iterations)) - len(str(i + 1)))
        print('[iteration %s%d]objective_train %0.3f | objective_test %0.3f | r %0.3f | b_node %0.3f | b_edge %0.3f | entropy %0.3f' %
              (prefix, i + 1, objective_train, objective_test, r, b_node, b_edge, entropy))

In [ ]:
# ckpt = eager.Checkpoint(model=network)
# ckpt.save('./models/network')